In [1]:
import socket
import time
import numpy as np  
from astropy.io import fits 

In [2]:

# Function to send a command
def send_command(sock, buf):
    sock.sendto(buf.encode(), sock_out)

# Function to start recording
def start_recording():
    global time_rec_start
    time_rec_start = time.localtime()

    date_start = "%02d-%02d-%02d" % (time_rec_start.tm_year, time_rec_start.tm_mon, time_rec_start.tm_mday)
    time_start = "%02d-%02d-%02d" % (time_rec_start.tm_hour, time_rec_start.tm_min, time_rec_start.tm_sec)
    time_end = "00-00-00"

    return date_start, time_start, time_end

# Function to stop recording
def stop_recording():
    time_rec_stop = time.localtime()

    if time_rec_stop.tm_sec < time_rec_start.tm_sec:
        time_rec_stop.tm_sec += 60
    if time_rec_stop.tm_min < time_rec_start.tm_min:
        time_rec_stop.tm_min += 60
    if time_rec_stop.tm_hour < time_rec_start.tm_hour:
        time_rec_stop.tm_hour += 24

    time_end = "%02d-%02d-%02d" % (time_rec_stop.tm_hour, time_rec_stop.tm_min, time_rec_stop.tm_sec)

    return time_end

# Function to receive data and create FITS file
def ask(param1):
    global num_pkt, i_free, data_sample_rcp_0, data_sample_lcp_0, data_sample_rcp_1, data_sample_lcp_1, data_sample_rcp_2, data_sample_lcp_2, data_sample_rcp_3, data_sample_lcp_3, data_sample_rcp_4, data_sample_lcp_4, ut_sample_rcp_0, ut_sample_lcp_0, ut_sample_rcp_1, ut_sample_lcp_1, ut_sample_rcp_2, ut_sample_lcp_2, ut_sample_rcp_3, ut_sample_lcp_3, ut_sample_rcp_4, ut_sample_lcp_4
    while True:
        recv_len = 0
        buf_sock_in = b'\0' * BUFSOCK  # Clear the buffer, it could have previously received data
        recv_len = s_in.recvfrom(BUFSOCK)
        if recv_len:
            # Parse the packet
            if rec_mozhno:
                # Allow writing to the output buffer
                # Parse the packet
                if buf_sock_in[0:2] == b'LN' and buf_sock_in[4] == b'S':
                    # Status packet
                    temp1 = buf_sock_in[9:13].decode()
                    temp2 = buf_sock_in[16:20].decode()
                    cool = COOL if buf_sock_in[29] == b'C' else HEAT
                if buf_sock_in[0:2] == b'LN' and buf_sock_in[4] == b'D':
                    # Data packet
                    num_pkt += 1
                    # Process data packet and fill the output buffer
                    if i_free < BUFLEN:
                        millis = [0] * 10
                        millis_base = 0
                        str_millis_base = buf_sock_in[8+4+16+4*(4+32):8+4+16+4*(4+32)+8].decode()
                        millis_base = int(str_millis_base, 16)
                        for i in range(5):
                            str_mill0 = buf_sock_in[8+4+i*(4+32):8+4+i*(4+32)+8].decode()
                            millis[2*i] = int(str_mill0, 16)
                            str_data0 = buf_sock_in[8+4+8+i*(4+32):8+4+8+i*(4+32)+8].decode()
                            data_sample_rcp_0[i_free] = int(str_data0, 16)
                            str_mill1 = buf_sock_in[8+4+16+i*(4+32):8+4+16+i*(4+32)+8].decode()
                            millis[2*i+1] = int(str_mill1, 16)
                            str_data1 = buf_sock_in[8+4+24+i*(4+32):8+4+24+i*(4+32)+8].decode()
                            data_sample_lcp_0[i_free] = int(str_data1, 16)
                        utime_base = now.tm_hour + now.tm_min/60. + now.tm_sec/3600. + now.tm_msec/3600000.
                        ut_sample_rcp_0[i_free] = utime_base - (millis_base - millis[0])/3600000.
                        ut_sample_lcp_0[i_free] = utime_base - (millis_base - millis[1])/3600000.
                        i_free += 1
                    else:
                        print("Buffer full, data skipped!")
                    send_command(s_out, "FL\n")
                # Send the command to empty the input buffer
                # if a data packet has been received
                if num_pkt >= PACKETS:
                    send_command(s_out, "FL\n")
                    num_pkt = 0
            else:
                send_command(s_out, "FL\n")
        else:
            # Create FITS file with collected data
            create_fits_file(data_sample_rcp_0[:i_free], data_sample_lcp_0[:i_free],
                             ut_sample_rcp_0[:i_free], ut_sample_lcp_0[:i_free])

# Function to create FITS file
def create_fits_file(data_sample_rcp_0, data_sample_lcp_0, ut_sample_rcp_0, ut_sample_lcp_0):
    # Create numpy arrays from the collected data
    data_rcp_0 = np.array(data_sample_rcp_0)
    data_lcp_0 = np.array(data_sample_lcp_0)

    ut_rcp_0 = np.array(ut_sample_rcp_0)
    ut_lcp_0 = np.array(ut_sample_lcp_0)

    # Stack the data into a single array
    data = np.stack((data_rcp_0, data_lcp_0, ut_rcp_0, ut_lcp_0), axis=-1)

    # Create a FITS header
    hdr = fits.Header()
    hdr['TELESCOP'] = 'ALMA'
    hdr['INSTRUME'] = 'ANTENNA'
    hdr['DATE-OBS'] = time.strftime('%Y-%m-%d', time.gmtime())
    hdr['TIME-OBS'] = time.strftime('%H:%M:%S', time.gmtime())

    # Create a FITS primary HDU with the data and header
    hdu = fits.PrimaryHDU(data, header=hdr)

    # Create an HDU list and add the primary HDU
    hdul = fits.HDUList([hdu])

    # Save the FITS file
    hdul.writeto('antenna_data.fits', overwrite=True)
    print("FITS file created successfully!")


In [3]:
BUFSOCK = 2048  # Maximum buffer length
PORT = 8888   # Port to listen for incoming data
SP_SERVER = "127.0.0.1"  # IP address of the receiving controller

# Variables
s_in = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Socket for receiving
s_out = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)  # Socket for sending

# Bind the socket to the port for receiving
s_in.bind(('0.0.0.0', PORT))

sock_out = (SP_SERVER, PORT)

date_start, time_start, _ = start_recording()
ask("ST")
duration = 120
end_time = time.time() + duration
while time.time() < end_time:
    ask(param1)
    time.sleep(1)
time_end = stop_recording()
print("Recording started at: " + date_start + " " + time_start + "\n")
print("Recording stopped at: " + date_start + " " + time_end + "\n")

NameError: name 'BUFSOCK' is not defined